# Planning Agent
## Context
* Scene Outline (trope) - `str`
* Scene Setting - `Dict[str, str]`
    * location, setting, exploration
* Character Descriptions - `List[CharacterSpecification]`
* Action History - `List[Dict[str, str]]`

Example:
```
# Setting:
class SceneSetting(BaseModel):
    location: str
    setting: str
    explanation: str

# Character
class CharacterSpecification(BaseModel):
    name: str
    gender: DEFINED_ASPECTS["Gender"]
    age: int
    dialogue_tone: str
    career: DEFINED_ASPECTS["Career"]
    personality_traits: List[PersonalityTrait]
    hobbies: List[DEFINED_ASPECTS["Skill"]]
    living_conditions: List[str]
    social_relationships: List[SocialRelationship]
    
# History
class CharacterAction(BaseModel):
    character: str
    action: str 
```
## tool
* Action Predictor: `predict_action(character_uid: str, context: str)`


In [1]:
import itertools
import json
import os
import random
from typing import Any, List
import yaml

from jinja2 import Template, StrictUndefined
from jinja2.exceptions import TemplateError

from openai import AsyncOpenAI
from pydantic import BaseModel, Extra, Field

# pydantic-ai
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel, OpenAIResponsesModelSettings
from pydantic_ai.providers.openai import OpenAIProvider

import tiktoken

from src.models import (
    CharacterSpecification,
    CharacterState,
    Character,
    EmotionalState,
    create_dynamic_enum
)

from config import settings

In [2]:
tokenizer = tiktoken.encoding_for_model("gpt-4o-mini")
def calculate_tokens(x):
    return len(tokenizer.encode(x))
calculate_tokens("hello world")

2

# 1. Prompt Test

## 1-1. Load Character Spec

In [3]:
## Scene Data
scene_uid = "2f2462c1-bdef-4089-b206-c47decd841f3"

with open(f"assets/simschat/scene/{scene_uid}.json", "r") as f:
    scene_dict = json.load(f)

print(json.dumps(scene_dict,indent=4))

## Character Data
character_uids = [
    scene_dict["character_states"]["character1_uid"],
    scene_dict["character_states"]["character2_uid"],
]

character_specs = {}
for character_uid in character_uids:
    with open(f"assets/simschat/characters/{character_uid}.json", "r") as f:
        character_spec_dict = json.load(f)
    character_spec = CharacterSpecification(**character_spec_dict)
    character_specs[character_uid] = character_spec

{
    "uid": "2f2462c1-bdef-4089-b206-c47decd841f3",
    "stage": 0,
    "scene": {
        "revised_trope": "Inside the bustling grandeur of the metropolitan fashion gala, Zephyr Orion, a 28-year-old jocular astronaut with a penchant for playful storytelling, encounters Vivienne LaRoux. Vivienne, also 28, exudes sophistication and an assertive demeanor as a renowned style influencer. Initially, she greets Zephyr's lighthearted banter with icy indifference, her mean streak surfacing sporadically. However, Zephyr's infectious humor gradually softens her edges, revealing a subtly receptive side. Their verbal dance, rich with lively exchanges, challenges both to reconsider their outlooks, Zephyr embracing Vivienne's world of high fashion while she discovers a brighter perspective in his social magnetism.",
        "utilized_reference_tropes": [
            "Defrosting Ice Queen",
            "SavvyGuyEnergeticGirl"
        ],
        "scene": {
            "location": "The Metropolitan Fa

In [4]:
print(character_spec.model_dump_json(indent=2))

{
  "name": "Vivienne LaRoux",
  "gender": "female",
  "age": 28,
  "dialogue_tone": "Vivienne's speech is assertive and confident, laced with stylish, high-end fashion terminologies. Her voice is smooth and melodic, suggesting sophistication. However, it can become sharp when she is being dismissive or unkind.",
  "career": "Style Influencer",
  "personality_traits": [
    {
      "trait": "Genius",
      "description": "Intellectually gifted, Vivienne is viewed as a genius and thrives on intellectual pursuits."
    },
    {
      "trait": "Noncommittal",
      "description": "Vivienne is known for frequently changing her mind on a whim."
    },
    {
      "trait": "Mean",
      "description": "Despite her sophisticated demeanor, Vivienne possesses a mean streak and a dismissive nature that has created some contention among her peers."
    }
  ],
  "hobbies": [
    "Acting",
    "Singing"
  ],
  "living_conditions": [
    "Vivienne resides in a luxurious apartment in a bustling metro

## 1-2. Load Template

In [5]:
with open('prompts/planning_agent.yaml', 'r') as file:
    action_predictor_prompt = yaml.load(file, Loader=yaml.FullLoader)

system_message = action_predictor_prompt['system']
user_template = Template(
    action_predictor_prompt['user'],
    undefined=StrictUndefined
)

In [6]:
user_contents = {
    "scene_outline": scene_dict["scene"]["revised_trope"],
    "scene_background": scene_dict["scene"]["scene"],
    "characters": {k: v.model_dump_json() for k,v in character_specs.items()},
    "history": []
}

user_message = user_template.render(**user_contents)

In [7]:
print(calculate_tokens(user_message))
print(user_message)

969
[Scene Description]
* Outline: Inside the bustling grandeur of the metropolitan fashion gala, Zephyr Orion, a 28-year-old jocular astronaut with a penchant for playful storytelling, encounters Vivienne LaRoux. Vivienne, also 28, exudes sophistication and an assertive demeanor as a renowned style influencer. Initially, she greets Zephyr's lighthearted banter with icy indifference, her mean streak surfacing sporadically. However, Zephyr's infectious humor gradually softens her edges, revealing a subtly receptive side. Their verbal dance, rich with lively exchanges, challenges both to reconsider their outlooks, Zephyr embracing Vivienne's world of high fashion while she discovers a brighter perspective in his social magnetism.
* Background:
  * location: The Metropolitan Fashion Gala
  * setting: An opulent event hall adorned with sparkling chandeliers and cutting-edge fashion displays.
  * explanation: In the midst of a vibrant and luxurious fashion gala, Zephyr Orion and Vivienne La

# 2. Initialize Agent

In [8]:
# Initialize OpenAI provider
print(settings.llm_model)

# Pydantic Model
provider = OpenAIProvider(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key
)
model = OpenAIModel(
    model_name=settings.llm_model,
    provider=provider
)

# OpenAI Client
aopenai_client = AsyncOpenAI(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key
)

gpt-4.1-nano


In [9]:
class CharacterAction(BaseModel):
    character_uid: str
    action: str
    targets: List[str]
    
agent = Agent(
    model=model,
    name="planning",
    output_type=CharacterAction,
    system_prompt = system_message
)

In [10]:
@agent.tool
async def predict_action(
    ctx: RunContext[None],
    character_uid: str,
    scene_context: str
):
    """Dummy predict_action function"""
    print("CHARACTER_UID:", character_uid)
    print("SCENE_CONTEXT:", scene_context)
    
    return {
        "action": "Walk towards Vivienne",
        "targets": [
            "00d66087-9b3b-46da-bd74-bf45cbe81d3c"
        ]
    }


In [11]:
result = await agent.run(user_message)

CHARACTER_UID: 35f0c56f-263d-42df-846c-e1833d8ca0ab
SCENE_CONTEXT: Inside the opulent fashion gala with sparkling chandeliers and modern fashion displays, Zephyr Orion is engaging in a lively, playful conversation with Vivienne LaRoux. Zephyr's humor and storytelling are gradually softening her initially icy demeanor, revealing her subtly receptive side. Zephyr enjoys connecting with diverse social circles, blending his love of space and humor with fashion and high society.


In [12]:
result

AgentRunResult(output=CharacterAction(character_uid='35f0c56f-263d-42df-846c-e1833d8ca0ab', action='Walk towards Vivienne', targets=['00d66087-9b3b-46da-bd74-bf45cbe81d3c']))

In [13]:
print(result.output.model_dump_json(indent=4))

{
    "character_uid": "35f0c56f-263d-42df-846c-e1833d8ca0ab",
    "action": "Walk towards Vivienne",
    "targets": [
        "00d66087-9b3b-46da-bd74-bf45cbe81d3c"
    ]
}
